# Stationary multi-armed bandit

In the stationary case the reward probability $p_{t, l}$ is fixed, hence $p_{t,l} = p_l$, $ \forall t \in \{1, T\}$. We will consider here the variant of the problem in which all but the best arm have the same reward probability $p=1/2$, and the reward probability of the 'best' arm is set as $p_{max} = p + \epsilon$, where $\epsilon \in \left(0, \frac{1}{2}\right]$.

In [1]:
# load packages
from jax import devices

import numpy as np
import jax.numpy as jnp
from jax import random, lax, nn, ops, vmap, jit

devices(backend='cpu')

[CpuDevice(id=0)]

In [2]:
# append the path to the code folder
import sys
sys.path.append('../../code/')

# define the generative process for rewards and outcomes with zero change probability
from environment import generative_process_swtch
rho = .0 # change probability
log_pj_j = jnp.log(np.array([[1 - rho, rho], [1, 0]]))

process = lambda *args: generative_process_swtch(*args, log_pj_j)

# import the learning rule
from learning_algos import learning_stationary as learning

In [4]:
from choice_algos import betaincinv, func_1, func_2, compute_x, betaincinv

compute_x(1 - 1e-16, 100.*jnp.ones(10), 1000. * jnp.ones(10))

betaincinv(100.*jnp.ones(10), 100.*jnp.ones(10), .5)

[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
[False False False False False False False False False False]


DeviceArray([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [ ]:
jnp.logical_or

In [ ]:
jnp.clip(p, a_min=0., a_max=1.)

## Bayesian inference

Given the constrain of this investigation to the Bernoulli bandits we will define the observation likelihood as 

$$ p(o_t|\vec{\theta}, a_t) = \prod_{k=1}^K \left[ \theta_k^{o_{t}}\left( 1- \theta_k \right)^{1-o_{t}} \right]^{\delta_{a_t, k}}  $$

where $a_t$ denotes the agent's choice at trial $t$. Given the Bernoulli likelihoods we will assume that both priors and posterior correspond to conjugate distribution, the Beta distribution. Hence, we can express the prior (before any observation is made) as a product of prior beliefs over different arms

$$ p(\vec{\theta}) = \prod_{k=1}^K \mathcal{Be}(\theta_k; \alpha_{0,k}, \beta_{0,k}) $$

where we assume that initial prior (before making any observations) corresponds to a uniform distribution, 
hence $\alpha_{0,k}, \beta_{0,k} = 1, \forall \: k$. Conjugacy of the prior, in stationary cases, allows us to define simple update rules corresponding to exact Bayesian inference, as

\begin{equation}
    \begin{split}
    \alpha_{t, k} &= \alpha_{t-1, k} + \delta_{a_t, k} o_t \\
    \beta_{t, k} &= \beta_{t-1,k} + \delta_{a_t, k} (1-o_t)
    \end{split}
\end{equation}

where the parameter update is performed only for a selected $k$th arm at trial $t$. Hence, in a sequential inference setup the posterior beliefs $p(\vec{\theta}|o_{t:1})$ at trial $t$, become the prior beliefs at the next trial $t+1$, and can be expressed as
\begin{equation}
    p(\vec{\theta}|o_{t:1}, a_{t:1}) = \prod_{k=1}^K \mathcal{Be}\left(\theta_k; \alpha_{t,k}, \beta_{t,k}\right) .
\end{equation}

## Action selection

### Thompson sampling

This form of action selection algorithm is derived from the i.i.d samples from the reward probability prior
at trial $t$, hence

$$a_t = \arg\max_k \theta^*_k, \qquad \theta^*_k \sim \mathcal{Be}(\theta_k; \alpha_{t-1, k}, \beta_{t-1, k})$$

An extension of this often found in the literature, specially on dynamic MABs, is called optimistic 
Thompson sampling, and is defined as 

$$a_t = \arg\max_k \max(\theta^*_k, \mu_{t-1,k}), \qquad \theta^*_k \sim \mathcal{Be}(\theta_k; \alpha_{t-1, k}, \beta_{t-1, k})$$

where the expected reward probability $\mu_{t-1, k} = \frac{\alpha_{t-1,k}}{\alpha_{t-1,k} + \beta_{t-1, k} }$
constrains the minimal value of the sample from the prior.


### Upper confidence bound (UCB)

Another classical algorithm of reinforcement learning with a decision rule defined as

\begin{equation}
    a_t = \left\{ \begin{array}{cc}
        \arg\max_k \left(m_{k, t-1} + \frac{\ln t}{n_{k, t-1}} + \sqrt{\frac{ m_{k, t-1} \ln t}{n_{k, t-1}}}\right) & \textrm{for } t>K \\
        t & \textrm{otherwise}
    \end{array}
    \right.
\end{equation}

where $m_{k, t-1} = \frac{\alpha_{t-1, k} - \alpha_0}{n_{t-1, k}}$, and $n_{k, t-1} = \alpha_{t-1,k} + \beta_{t-1,k} - \alpha_0 - \beta_0$. Here, we will also consider a Bayesian variant of the upper confidence bound, in which the best arm is selected as the one with the highest $z$th percentile of posterior beliefs, where the percentile increases over time as $z_t = 1 - \frac{1}{t} $. Hence, 

\begin{equation}
    a_t = \arg\max_k CDF^{-1} \left( z_t, \alpha_{t-1}^k, \beta_{t-1}^k \right).
\end{equation}

In the case of the beta distributed beliefs $\mathcal{Be}\left(\alpha, \beta\right)$, the inverse cumulative distribution $CDF^{-1}$ corresponds to the inverse incomplete regularised beta function $I^{-1}_z \left( \alpha, \beta \right)$.


### Active inference

The action selection in active inference rest upon the expected free energy $G(\pi)$ of behavioral policy $\pi$. Normally, behavioral policies in active inference correspond to a specific sequence of future actions $\pi = (a_{t}, \ldots, a_D)$ up to some planning depth $D$. Here we will limit the analysis to a shallow planning depth of $D=1$ hence each policy corresponds to one of the possible choices, that is actions $a_t$.
The expected free energy is defined as 

\begin{equation}
    \begin{split}
    G_t(a) & =  D_{KL}\left(Q(o_t |a)||P(o_t)\right) + E_{Q(\vec{\theta}|a)}\left[H[o_t|\vec{\theta}, a] \right] \\
    & = - E_{Q(o_t|a)}\left[ \ln P(o_t) +  D_{KL}\left( Q(\vec{\theta}|o_t, a)|| Q(\vec{\theta})\right) \right]
    \end{split}
\end{equation}


where 

$$Q(o_t|a) = \int d \vec{\theta} p \left(o_t|\vec{\theta}, a\right) Q(\vec{\theta}),$$ 

and

$$ Q(\vec{\theta}|o_t, a) \propto p(o_t|\vec{\theta}, a) Q(\vec{\theta}).$$

Teh quantity $P(o_t)$ denotes prior preferences over outcomes, and here we set it in a way that the agent always prefers rewards (o_t=1) over no-rewards (o_t=0. We can express this as 

$$ P(o_t) = \propto e^{o_t \lambda} e^{-(1-o_t) \lambda},$$

where $\lambda$ parametrises strengt of preferences of rewards over no-rewards. Given the expected free energy the action selection rule is defined as 

$$ a_t = \arg\min_a G_t(a).$$

Given the known functional expressions for the prior expectation $Q\left(\vec{\theta}\right)$ as a product of Beta distributions, we get the following expressions for expected free energy 

\begin{equation}
    \begin{split}
    G_t(a) = & - \lambda( 2 \cdot  \mu_{t-1, a} - 1) + \mu_{t-1, a} \ln \mu_{t-1, a} + (1-\mu_{t-1, a}) \ln ( 1- \mu_{t-1, a}) \\
    & - \mu_{t-1,a} \psi(\alpha_{t-1, a}) - (1 - \mu_{t-1,a}) \psi(\beta_{t-1, a}) + \psi(\nu_{t-1,a}) - \frac{1}{\nu_{t-1,a}} \\
    \approx & -\lambda(2 \mu_{t-1, a} - 1) - \frac{1}{2 \nu_{t-1, a}} \equiv \tilde{G}_t(a)
    \end{split}
\end{equation}

Interestingly, the approximate form of the expected free energy allows us to express the expected epistemic affordance of each arm as (approximately)
    \begin{equation}
    \begin{split}
        E_{Q(o_t|a)}\left( D_{KL}\left[Q(\vec{\theta}|o_t, a)||Q(\vec{\theta})\right] \right) &= E_{Q\left(o_t, \vec{\theta}|a\right)}\left( \ln \frac{Q(\vec{\theta}| o_t, a_t)}{Q(\vec{\theta})} \right) \approx \frac{1}{2 \nu_{t-1, a_t}}
    \end{split}
    \end{equation}
    
Hence, the information gain of a particular choice $a$ and an outcome $o_t$ becomes

\begin{equation}
    D_{KL}\left[Q(\vec{\theta}|o_t, a)||Q(\vec{\theta})\right] = \left\{ \begin{array}{ll} - \ln \mu_{t-1}^{a} + \psi(\alpha_{t-1}^{a}+1) - \psi(\nu_{t-1}^{a}+1), & \textrm{ for } o_t=1 \\ - \ln \left( 1 - \mu_{t-1}^{a}\right) + \psi(\beta_{t-1}^{a}+1) - \psi(\nu_{t-1}^{a}+1), & \textrm{ for } o_t = 0 \end{array} \right. \approx \left\{ \begin{array}{ll} \frac{1}{2\nu_{t-1}^{a}} \left( \frac{1}{\mu_{t-1}^{a}} - 1\right), & \textrm{ for } o_t=1 \\ \frac{1}{2\nu_{t-1}^{a}} \left( \frac{1}{1 - \mu_{t-1}^{a}} - 1\right), & \textrm{ for } o_t = 0 \end{array}\right. 
\end{equation}


In [3]:
# import decision algorithms
from choice_algos import thompson_selection, ots_selection, ucb_selection, bucb_selection, efe_selection, app_selection, ai_sampling_selection

In [4]:
# implements the simulator for POMDP


In [5]:
# implements regret rate estimation over different AI algos difficulty levels
from collections import defaultdict
from tqdm.notebook import trange
import itertools

def estimate_regret_rate1(Ks, epsilon, steps=1000, N=1000):
    regret_all = defaultdict(lambda: [])

    seed = random.PRNGKey(100)
    lambdas = jnp.arange(.0, 2., .05)
    gammas = jnp.ones(1) * 1000.
    
    for i in trange(len(Ks), desc='K loop'):
        K = Ks[i]
        for func, label in zip([efe_selection, sup_selection, app_selection], 
                       ['EFE_K{}'.format(K), 'SUP_K{}'.format(K), 'APP_K{}'.format(K)]):

            seed, _seed = random.split(seed)
            sim = lambda e, g, l: simulator(process, 
                                            learning, 
                                            lambda *args: func(*args, gamma=g, lam=l), 
                                            N=N, steps=steps, K=K, 
                                            eps=e, 
                                            seed=_seed[0])
            
            vals = jnp.array(list(itertools.product(gammas, lambdas)))
            
            cum_regret, cum_epst_regret = vmap(lambda g, l: sim(epsilon, g, l))(vals[:, 0], vals[:, 1])
            regret_all[label].append({'regret': cum_regret, 'epistemics': cum_epst_regret})
            
    np.savez('res_AI_Ks_e{}'.format(int(epsilon * 100)), **regret_all)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate1(Ks, epsilon=.1)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate1(Ks, epsilon=.25)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate1(Ks, epsilon=.4)

In [6]:
def estimate_regret_rate2(Ks, epsilon, steps=1000, N=1000):
    mean_reg = defaultdict(lambda: [])

    seed = random.PRNGKey(100)

    for i in trange(len(Ks), desc='K loop'):
        K = Ks[i]
        for func, label in zip([thompson_selection, ots_selection, ucb_selection], 
                       ['TS_K{}'.format(K), 'OTS_K{}'.format(K), 'UCB_K{}'.format(K)]):

            seed, _seed = random.split(seed)

            cum_regret, cum_epst_regret = simulator(process, 
                                                    learning, 
                                                    func, 
                                                    N=N, steps=steps, K=K, 
                                                    eps=epsilon,
                                                    seed=_seed[0])
                
            mean_reg[label].append({'regret': cum_regret, 'epistemics': cum_epst_regret})
    
    np.savez('res_RL_Ks_e{}'.format(int(epsilon * 100)), **mean_reg)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate2(Ks, epsilon=.1)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate2(Ks, epsilon=.25)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate2(Ks, epsilon=.4)

In [13]:
# implements the simulator for POMDP using bucb selection
# inverse incomplete beta is not a jax function, so the code has to be in pure python
def fori_loop(low, up, func, carry):
    for i in range(low, up):
        carry = func(i, carry)
    return carry

def scan(func, carry, iterate):
    saves = []
    for n in iterate:
        carry, res = func(carry, n)
        saves.append(res)
    
    return np.stack(saves, 0)

def simulator_bucb(process, learning, action_selection, N=1000, T=1000, K=10, seed=0, eps=.25, save_every=100):
    def loop_fn(t, carry):
        rng_key, states, prior, cum_reg, cum_epst_reg = carry

        rng_key, _rng_key = random.split(rng_key)
        choices = action_selection(t, prior, _rng_key)

        rng_key, _rng_key = random.split(rng_key)
        outcomes, states = process(t, choices, states, _rng_key)
        posterior = learning(outcomes, choices, prior)

        sel = jnp.arange(N)

        alphas = prior[sel, choices, 0]
        betas = prior[sel, choices, 1]
        nu = alphas + betas
        mu = alphas/nu

        nu_min = jnp.min(prior[..., 0] + prior[..., 1], -1)
        cum_reg += eps * ~(choices == 0)

        cum_epst_reg += (1/nu_min - 1/nu)/2

        return (rng_key, states, posterior, cum_reg, cum_epst_reg)
    
    def sim_fn(carry, t):
        res = fori_loop(t * save_every, (t+1) * save_every, loop_fn, carry)
        _, _, _, cum_reg, cum_epst_reg = res
        return res, (cum_reg, cum_epst_reg)
    
    rng_key = random.PRNGKey(seed)
    probs = np.concatenate([np.array([eps + .5]), np.ones(K-1)/2.])
    states = [probs, np.zeros(1, dtype=np.int32)]
    prior = np.ones((N, K, 2))
    
    results = scan(sim_fn, (rng_key, states, prior, np.zeros(N), np.zeros(N)), np.arange(T))
    
    return results

def estimate_regret_rate_bucb(Ks, epsilon, T=10000, N=500):
    mean_reg = defaultdict(lambda: [])

    seed = random.PRNGKey(100)
    times = np.arange(1, T+1)[::10, None]

    for i in trange(len(Ks), desc='K loop'):
        K = Ks[i]
        for func, label in zip([bucb_selection], 
                       ['BUCB_K{}'.format(K)]):

            seed, _seed = random.split(seed)
            res = simulator_bucb(process, 
                                 learning, 
                                 func, 
                                 N=N, T=T, K=K, 
                                 eps=epsilon,
                                 seed=_seed[0])
            
            regret = res[:, 0]
            info_gain = res[:, 1]
                
            cum_regret = np.cumsum(regret.astype(jnp.float32), -2)[::10]
            cum_ig = np.cumsum(info_gain.astype(jnp.float32), -2)[::10]
            mean_reg[label].append({'regret': cum_regret, 'epistemics': cum_ig})

    np.savez('res_BUCB_Ks_e{}'.format(int(epsilon * 100)), **mean_reg)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate_bucb(Ks, epsilon=.1)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate_bucb(Ks, epsilon=.25)

In [ ]:
Ks = [5, 10, 20, 40, 80]
estimate_regret_rate_bucb(Ks, epsilon=.4)

### TODO:

* establish functional or numerical relationship between $\lambda^*$ and $K$, $\epsilon$.
* determine the optimality relation for AI algorithms $\lambda^* = f(K, \epsilon)$? 
* introduce learning o $\lambda$. Would the learning find values of $\lambda$ that minimize regret?